### DATA 612 - Project #5

Zach Alexander  
7/4/2020

### Instructions

Adapt one of your recommendation systems to work with Apache Spark and compare the performance with your previous iteration. Consider the efficiency of the system and the added complexity of using Spark. You may complete the assignment using PySpark (Python), SparkR (R), sparklyr (R), or Scala. Please include in your conclusion: For your given recommender system’s data, algorithm(s), and (envisioned) implementation, at what point would you see moving to a distributed platform such as Spark becoming necessary?

##### Declaring time-tracking functions to measure performance in PySpark 

Before diving into the work, in order to measure and compare the performance of my work in Databricks (utilizing PySpark), relative to my work in R in previous weeks, I decided to load in two functions `tic()` and `toc()`. These will help set up a time window to measure the time elapsed from the beginning to the end of a certain piece of code. I'll use this throughout, but wanted to make sure I have these ready to go for my work in the following lines.

In [4]:
def tic():
    #Homemade version of matlab tic and toc functions
    import time
    global startTime_for_tictoc
    startTime_for_tictoc = time.time()

def toc():
    import time
    if 'startTime_for_tictoc' in globals():
        print("Elapsed time is " + str(time.time() - startTime_for_tictoc) + " seconds.")
    else:
        print("Toc: start time not set")

### Loading in Movies Data

Similar to how our professor walked through this process in our lecture this past week, I'll take a look at the MovieLens data, which I also did extensive analysis on the first few weeks of the course. Given that I ran multiple algorithms in past projects, and built recommender systems based on these ratings, I'll be able to see if an algorithm that I run on PySpark will perform better than my previous iterations in R.

##### Saving the movies data as a csv file

First, I saved the movies data as a separate csv file, and loaded it into the "Data" section of Databricks. After assigning it with a file type of .csv, indicating to read the first row as a header, and delimiter as a comma, I then created a Spark dataframe and saved it as `movies_df`. I then displayed this below:

In [7]:
# File location and type
movies_file_location = "/FileStore/tables/movies-2.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
movies_df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(movies_file_location)

# display(movies_df)

### Loading in the Ratings Data

With the movies data loaded in as a Spark dataframe, I then did the same process for the ratings data. Since we'll be utilizing the ratings data to build our model later, we also need to define a schema, indicating the data type for each of the four columns. I saved the schema as `movies_schema`, saved the ratings data in a dataframe, `ratings_df`, and displayed it below:

In [9]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType

ratings_file_location = "/FileStore/tables/ratings-2.csv"

movie_schema = StructType([
  StructField("userId", IntegerType()),
  StructField("movieId", IntegerType()),
  StructField("rating", DoubleType()),
  StructField("timestamp", DoubleType())
])

# The applied options are for CSV files. For other file types, these will be ignored.
ratings_df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .schema(movie_schema) \
  .load(ratings_file_location)

# display(ratings_df)

We can also take a quick look at the dimensions of the dataframe:

In [11]:
print("Number of rows in ratings_df dataset : {}".format(str(ratings_df.count())))
print("Number of columns in ratings_df dataset : {}".format(str(len(ratings_df.columns))))

Number of rows in ratings_df dataset : 1000209
Number of columns in ratings_df dataset : 4

### Splitting into Training and Testing Datasets

Now, with our ratings and movies data loaded into Databricks successfully, I then was able to split the full `ratings_df` into a training and test dataset. I decided to do a split of 80/20, similar to past weeks. In the end, we can see the dimensions of our training and test datasets below:

In [13]:
(training, test) = ratings_df.randomSplit([0.8, 0.2])

In [14]:
print("Number of rows in training dataset : {}".format(str(training.count())))
print("Number of columns in training dataset : {}".format(str(len(training.columns))))
print("Number of rows in test dataset : {}".format(str(test.count())))
print("Number of columns in test dataset : {}".format(str(len(test.columns))))

Number of rows in training dataset : 800290
Number of columns in training dataset : 4
Number of rows in test dataset : 199919
Number of columns in test dataset : 4

In [15]:
from pyspark.ml.recommendation import ALS

tic()
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")
toc()

Elapsed time is 0.16474390029907227 seconds.

In [16]:
tic()
model = als.fit(training)
toc()

Elapsed time is 15.110301971435547 seconds.

In [17]:
tic()
predictions = model.transform(test)
toc()

Elapsed time is 0.02061295509338379 seconds.

In [18]:
# display(predictions)

In [19]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(
    metricName="rmse", labelCol="rating", 
    predictionCol="prediction")

rmse = evaluator.evaluate(predictions)

In [20]:
# Evaluation results
print("RMSE : {}".format(str(rmse)))

RMSE : 0.8948988851599775

### Assessing performance

After running the matrix factorization method above (ALS), and measuring the elapsed time from fitting the training data to the model as well as the time it took to make the predictions, we can see the following when we compare these processing times to our SVD factorization method in R in past weeks:

In [22]:
import pandas as pd

performance_table = pd.DataFrame([['PySpark', "{:.2f}".format(14.17) + " seconds", "{:.2f}".format(0.02) + " seconds"],
             ['R', "{:.2f}".format(1.7) + " seconds", "{:.2f}".format(0.06) + " seconds"]],
            columns = ['Platform', 'Fitting Training Data to Model/Computations', 'Making Predictions'])

performance_table

,Platform,Fitting Training Data to Model/Computations,Making Predictions
0,PySpark,14.17 seconds,0.02 seconds
1,R,1.70 seconds,0.06 seconds


Interestingly, we can see that R seems to perform faster during the time it takes to fit the training data to the matrix factorization model. However, we see a noticeable difference in speed in making predictions between PySpark and R. PySpark was able to make predictions on the full ratings dataset about **3 times faster** than our predictions in R.

### Second Algorithm Test -- Attempting to run SVD in PySpark and comparing to R

After reading a fair bit of documentation, it seems like it's not very straightforward to run SVD in PySpark. However, I'm going to attempt to do it here and measure its performance against the process I outlined in my Project #3 assignment a few weeks ago. Although I've already seen that running a similar matrix factorization method (ALS) above is much faster in PySpark, I did want to see if I could do an "apples to apples" comparison from one platform to the other.

Therefore, I initially decided to load in my user-movie matrix that I developed in R from Project #3. This matrix has already been imputed to have its missing values recalculated to the row mean. Therefore, this should be all set to run through SVD here in PySpark.

Below, you can see that I've successfully loaded in this user-movie matrix and assigned it to a spark dataframe of `R_movie_matrix`:

In [25]:
file_location = "/FileStore/tables/movie_matrix-1.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
R_movie_matrix = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

# display(R_movie_matrix)

We can confirm that this matrix has the same rows and columns as our matrix in R.

In [27]:
print("Number of rows in R_movie_matrix dataset : {}".format(str(R_movie_matrix.count())))
print("Number of columns in R_movie_matrix dataset : {}".format(str(len(R_movie_matrix.columns))))

Number of rows in R_movie_matrix dataset : 943
Number of columns in R_movie_matrix dataset : 1664

Next, we need to convert our spark dataframe into a RDD RowMatrix format in order to do matrix multiplication on it later. Therefore, I mapped through the dataframe and created a RowMatrix:

In [29]:
from pyspark.mllib.linalg import Vectors, DenseMatrix
from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix, RowMatrix

rdd = R_movie_matrix.rdd.map(list)
mat = RowMatrix(rdd)


With our RowMatrix ready to go, I then was able to use the `computeSVD()` function in PySpark to run singular value decomposition with our k value equal to 20, which was the same value as my optimal SVD computations in R in Project #3. The goal here is to obtain the same eigen values for s:

In [31]:
svd = mat.computeSVD(20, computeU=True)
U = svd.U
s = svd.s
V = svd.V

After running SVD on our matrix, we can indeed see that we obtain the same eigenvalues for s:

In [33]:
print("Singular values are: %s" % s)

Singular values are: [4528.795310209989,65.74346656442125,50.424327505714935,38.55470593522779,37.13876053402636,35.776545081960975,33.91172748271665,32.676419114502984,30.791885251212594,29.471643259235048,29.299125195958123,28.172323603732355,27.891424686553776,27.834347740275255,27.523725583591165,27.25689130512897,26.898323532598972,26.732033504109708,26.339615863951018,26.315940258488915]

I then was able to save my U, s, and V variables, and created a sigma variable below that has the eigenvalues of s oriented in a diagonal matrix. I was also able to compute the transpose of V. I created two new DenseMatrix variables in order to do this.

In [35]:
import numpy as np

sigma = DenseMatrix(len(s), len(s), np.diag(s).ravel("F"))

V_transpose = DenseMatrix(V.numCols, V.numRows, V.toArray().transpose().ravel("F"))

With the new values computed and saved in proper PySpark formats, I was able to multiply the U orthonormal matrix by the sigma diagonal matrix by the V orthonormal matrix to obtain my SVD matrix with ratings predictions.

In [37]:
mat_ = U.multiply(sigma).multiply(V_transpose)

As we can see below, our new `mat_` matrix has the same dimensions as our original matrix we started with, which was to be expected. However, `mat_` should have our new prediction ratings computed for all user-movie combinations.

In [39]:
print(mat_.numCols())
print(mat_.numRows())

1664
943

After doing a quick check on my `movie_matrix_svd` matrix that I computed in R for Project #3, I'm able to calculate the same prediction ratings here in PySpark. We can confirm by looking at the values of the first user, which are indeed the same as the user-predicted ratings for my Project #3 dataframe in R.

In [41]:
mat_.rows.first()

Out[23]: DenseVector([4.3125, 3.6207, 3.3733, 3.5236, 3.2783, 3.7232, 4.6954, 3.9236, 4.4911, 3.7586, 3.3358, 4.4699, 4.2959, 4.1518, 3.7446, 3.7951, 3.2872, 3.6105, 3.6634, 3.6418, 3.089, 3.6511, 3.6332, 3.8188, 4.0597, 3.4801, 3.4696, 3.9281, 2.7461, 3.648, 3.7341, 3.7167, 3.5756, 3.4375, 3.2689, 3.52, 3.4125, 3.2084, 3.5867, 3.5844, 3.3074, 3.8738, 3.5558, 3.5316, 3.9259, 3.7229, 3.3587, 4.2922, 3.5675, 6.1129, 3.6564, 3.8527, 3.5882, 3.4971, 3.6998, 4.6262, 3.7444, 4.1034, 4.0697, 4.0725, 3.8666, 3.8587, 3.2086, 4.0968, 3.696, 3.5315, 2.9933, 3.5075, 3.1521, 3.7775, 3.4479, 3.6162, 3.4477, 3.3748, 3.5951, 3.517, 3.5605, 3.0076, 3.907, 3.4685, 3.6029, 3.8139, 4.1928, 3.6329, 3.3725, 3.6628, 4.0963, 3.2808, 4.9756, 3.5387, 3.9126, 3.7453, 3.7994, 2.9252, 3.575, 4.6667, 3.4569, 4.2363, 3.2097, 5.7657, 3.4922, 3.4315, 3.1877, 3.3137, 3.2458, 3.5854, 3.7601, 3.6746, 4.1478, 3.3467, 4.0255, 3.2846, 3.7163, 4.1739, 3.6946, 3.6818, 3.5937, 2.9805, 3.7315, 3.1, 3.8342, 3.1284, 3.5344, 4.4217, 3.3201, 3.7643, 4.2876, 3.7392, 4.0137, 3.429, 3.2623, 3.53, 3.5769, 4.2008, 4.1415, 3.6845, 4.4645, 3.1266, 3.4084, 2.992, 3.4659, 3.3409, 2.6779, 4.1951, 3.1969, 3.4828, 3.3957, 3.13, 3.5639, 4.0944, 3.5643, 4.0732, 4.3921, 4.217, 2.9404, 3.7708, 3.8651, 3.123, 3.2351, 3.5779, 3.5749, 3.283, 3.819, 3.9179, 3.8491, 3.9333, 3.3576, 4.7677, 4.9377, 4.1097, 4.016, 5.2659, 5.4315, 5.4305, 4.1141, 4.593, 3.9236, 4.0654, 3.6679, 3.3968, 5.3663, 3.7418, 4.7849, 4.082, 3.6762, 4.2253, 3.6744, 3.6632, 4.1705, 4.0956, 4.1477, 3.4258, 3.73, 4.2364, 4.3623, 4.2516, 3.7301, 4.0261, 4.1747, 3.3487, 3.7251, 4.1982, 3.529, 4.6041, 3.4519, 3.6774, 3.8928, 4.3369, 3.9823, 4.437, 4.2953, 3.9948, 3.7012, 3.5395, 3.2427, 3.9432, 3.391, 3.192, 2.995, 3.3283, 3.8216, 4.0011, 4.4183, 3.8823, 2.673, 3.546, 3.6157, 4.7082, 3.6188, 3.9307, 2.6769, 3.3136, 3.4274, 3.6152, 3.1831, 3.7375, 3.4899, 3.9955, 3.8702, 2.7947, 3.5737, 3.6491, 2.6623, 3.3663, 2.9253, 4.0781, 3.2919, 4.077, 3.4492, 3.7495, 3.8465, 3.3167, 3.529, 3.2142, 3.3421, 3.6234, 4.1467, 5.3402, 3.2147, 3.1874, 2.9448, 3.7683, 3.3109, 3.3537, 4.1552, 3.1743, 3.7107, 3.8728, 3.7375, 3.474, 3.8866, 3.6048, 3.5619, 4.2419, 5.075, 3.6372, 3.2696, 3.6436, 3.4732, 3.2781, 3.4101, 3.7753, 3.1669, 4.1351, 2.9777, 3.4585, 3.6188, 3.2273, 3.6352, 3.3449, 3.5049, 3.7491, 3.691, 3.6075, 3.5948, 3.7677, 3.78, 3.3624, 3.4206, 3.4971, 4.1123, 3.7372, 3.633, 3.6017, 3.4674, 3.6639, 3.5286, 3.6614, 3.6496, 3.3065, 3.7273, 3.4643, 3.6951, 3.6272, 3.8448, 3.9568, 3.7065, 3.5333, 3.5507, 2.9916, 3.6175, 3.8484, 3.4864, 3.509, 3.6899, 3.2892, 3.5418, 3.5128, 3.5484, 3.4069, 3.6297, 3.4227, 3.642, 3.4028, 3.5732, 3.0091, 3.5178, 3.4602, 3.5727, 3.589, 3.3908, 3.4978, 3.7523, 3.6485, 3.6564, 3.4546, 3.5669, 3.5279, 3.6689, 3.6528, 3.5763, 3.6029, 3.2623, 3.537, 3.2566, 3.5818, 3.6362, 3.5839, 3.6588, 3.6985, 3.4435, 3.539, 3.4252, 3.6553, 3.0645, 3.153, 3.3469, 3.5094, 3.7285, 3.5252, 3.4642, 3.2538, 3.3624, 3.3669, 3.1457, 3.2352, 3.6889, 3.6258, 3.8448, 3.5028, 3.3707, 3.6899, 3.2387, 3.4635, 3.2268, 3.4849, 3.7329, 3.565, 3.4074, 2.956, 3.5247, 3.2183, 3.3994, 3.462, 3.5158, 3.7687, 3.7139, 3.5166, 3.1758, 3.5507, 3.2424, 3.3472, 3.4191, 3.5258, 4.5797, 3.2505, 2.888, 3.1694, 2.9463, 3.5363, 3.7012, 3.5513, 3.4004, 3.2211, 2.9795, 2.9936, 3.3553, 3.8487, 3.5587, 3.683, 3.1655, 3.7751, 3.4701, 3.6108, 3.9936, 3.8414, 4.1504, 3.4804, 3.4529, 4.1613, 3.883, 4.0351, 3.9335, 3.4412, 3.4378, 3.4412, 3.3501, 3.2369, 3.4704, 3.7231, 3.4497, 3.5705, 3.6114, 3.4941, 3.2683, 3.6972, 3.3386, 3.077, 3.0074, 3.28, 3.5562, 4.1557, 3.0522, 3.0093, 3.4758, 3.5894, 3.6861, 3.6014, 3.984, 3.7573, 3.7423, 3.43, 3.7794, 3.6943, 3.4246, 3.8053, 3.4341, 3.5539, 2.8028, 3.2901, 4.1694, 3.8976, 3.0209, 3.7297, 3.8422, 4.1077, 3.7106, 3.5606, 3.7452, 4.266, 4.0319, 3.2484, 3.6876, 3.6624, 3.5527, 3.7038, 3.6166, 3.8278, 3.6836, 3.7845, 3.6635, 3.5206, 3.7215, 3.72, 3.9188, 3.6557, 3.1479, 3.8968, 3.6246, 3.6866, 3.6615, 3.6761, 3.4819, 4.1714, 3.9935, 

### Assessing performance of SVD in PySpark and R

After doing this apples-to-apples comparison between the two platforms, we can see the following processing time breakdown:

In [43]:
performance_table = pd.DataFrame([['PySpark SVD', "{:.2f}".format(502) + " seconds", "{:.2f}".format(0.07) + " seconds"],
             ['R SVD', "{:.2f}".format(1.7) + " seconds", "{:.2f}".format(0.06) + " seconds"]],
            columns = ['Platform/Method', 'Computing SVD', 'Making Predictions'])

performance_table

,Platform/Method,Computing SVD,Making Predictions
0,PySpark SVD,502.00 seconds,0.07 seconds
1,R SVD,1.70 seconds,0.06 seconds


### Conclusion

After working through both ALS and SVD in PySpark, I've found a few takeaways:  

+ ALS in PySpark seemed to perform about **three times faster** than SVD in R when looking strictly at its ability to make ratings predictions  
+ ALS in PySpark was much slower than R, about 14 seconds to 2 seconds, in terms of fitting the training data to the model and computing singular values in R (although this is not a true comparison, so doesn't hold much weight)  

Although we cannot really draw too many concrete conclusions from this initial comparison, it does look like PySpark may be more efficient at making predictions once the data has been processed and/or fit to the training data. When we ran the SVD technique in PySpark as well, and compared this to the same computations from Project #3, this seems to confirm these findings:  

+ SVD computation in PySpark was *very* slow, taking about 502 seconds (~8.5 minutes) with a k-value of 20. This was much slower than the approximately 2 seconds it took to run this function in R.  
+ However, once SVD had been computed in PySpark, it seemed to make predictions on our user-movie matrix **at about the same efficiency** as we saw in R (about 0.06 seconds for each).  


##### Envisioned Implimentation
This makes me consider a few things about implimentation -- 1) Using SVD in PySpark may not be the most efficient technique to use for matrix factorization and dimensionality reduction. If we were to role this out to production, we'd seriously want to think through the implications of using a technique that takes about 8.5 minutes to process a relatively small dataset of ratings. If SVD did yield the best predictions and was chosen to role out to production, it would be important to think about how this system would handle inputs of new ratings. Likely, it would be advantageous to run updates to the predictions (based on new data) in batches, in order to prevent slow processing times on the front-end of an application. However, if it were possible to use ALS as a good alternative to SVD, it may be more worthwhile to do so. 2) PySpark seems to do a good job of making predictions and utilizing the data it has available once it has been processed. Although we saw slower processing times in PySpark than we did in R, the efficiencies of making faster predictions may be more beneficial when thinking about implementation. As mentioned above, batch updates could help keep the models up to date as much as possible, while still having the advantages of faster processing time for predictions. In the end, it would be important to consider the pros and cons to these two factors in determining which algorithms, techniques and platforms to use to implement a recommender system in production.  

##### Transitioning to Distributed Platform
I could definitely see it becoming necessary to transition to a distributed platform when the volume of data reaches into the millions and billions. Given that PySpark would be able to handle computations in a distributed fashion, eventually the larger your data volume, the more efficient it'll be. For our example utilizing the MovieLens data (and doing computations over a thousand rows), we didn't see the full advantages of Spark. However, once an application or system is scaled to meet the demand of a much larger volume of users/clients (i.e. millions/billions, big data, etc.), it would not be possible to process these consuming algorithms and techniques in a non-distributed fashion.

##### Added Complexity of Using Spark
I will say that it was a bit tricky to get acclimated to Spark this week. The resources are helpful, but at this point in time, it's difficult for me to fully wrap my head around the nuances of distributed data types such as RowMatrices, Spark dataframes, etc., and how they operate differently from something like a Pandas dataframe. With time, I'm sure these types of things become more familiar, but for those running algorithms/machine learning techniques on smaller datasets, I can definitely see why some data scientists tend to work in a non-distributed system to save on time, confusion (and sometimes efficiency).